In [0]:

from datetime import timedelta, date, datetime
from random import randint
import psycopg2
from contextlib import closing


dbinfo = {
    'NAME': 'cohort',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': 'internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com',
    'PORT': 7432,
    
}
conn = psycopg2.connect("""
            dbname={dbname} user={user} host={host} port={port} password={passwd}
            """.format(dbname=dbinfo['NAME'], user=dbinfo['USER'], passwd=dbinfo['PASSWORD'],
                       host=dbinfo['HOST'], port=dbinfo['PORT']))

def select(stmt, params):
    with closing(conn.cursor()) as cur:
        cur.execute(stmt, params)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt


from aaintdatapipeline.application.app_qa.common.utils import get_random_day_in_last_month, datetime_to_string, \
    get_random_day_in_last_week, get_week_start_end_date, send_data_source_date_email, get_month_start_end_date, \
    string_to_datetime

# get date
# today = date.today()
# date = get_random_day_in_last_week(today)
# start_date, end_date = get_week_start_end_date(date)
# if today.isoweekday() < 6:
#     end_date = (string_to_datetime(start_date) - timedelta(days=1)).date()


# start_date, end_date = get_month_start_end_date(get_random_day_in_last_month(datetime_to_string(today)))
# if today.day <= 6:
#     end_date = (string_to_datetime(start_date) - datetime.timedelta(days=1)).date()

# print(end_date)

# sql test
# mb_countries_list = ['au', 'br', 'ca', 'de', 'es', 'fr', 'gb', 'it', 'jp', 'kr', 'mx', 'ru', 'tr', 'tw', 'us']
# for device_code in ['ip', 'ap']:
#     for country_code in mb_countries_list:
date='2019-11-09'
granularity='w'
mw_led = """select sum(cnt) from plproxy.execute_select($$select count(1) as cnt from mw.domain_latest_est_date "
"where date = '{}' and granularity = '{}' and device_code = 'ip'$$)t(cnt bigint);""".format(date, granularity)
rows, _ = select(mw_led, None)
print rows

date='2019-10-31'
granularity='m'
mw_led = """select sum(cnt) from plproxy.execute_select($$select count(1) as cnt from mw.domain_latest_est_date "
"where date = '{}' and granularity = '{}' and device_code = 'ip'$$)t(cnt bigint);""".format(date, granularity)
rows, _ = select(mw_led, None)
print rows
# rows, _ = select(daily, None)
#weekly_rows, _ = select(weekly_daily, None)


In [0]:

import datetime
from random import randint
from aaintdatapipeline.application.app_qa.common.utils import get_random_day_in_last_month, datetime_to_string, \
    get_random_day_in_last_week, get_week_start_end_date, send_data_source_date_email, get_month_start_end_date, \
    string_to_datetime
    
import psycopg2
from contextlib import closing


dbinfo = {
    'NAME': 'cohort',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': 'internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com',
    'PORT': 7432,
    
}
conn = psycopg2.connect("""
            dbname={dbname} user={user} host={host} port={port} password={passwd}
            """.format(dbname=dbinfo['NAME'], user=dbinfo['USER'], passwd=dbinfo['PASSWORD'],
                       host=dbinfo['HOST'], port=dbinfo['PORT']))
                       
mb_device_mapping = {
    'it': 'iOS-tablet',
    'ip': 'iOS-phone',
    'ia': 'iOS-all',
    'at': 'Android-tablet',
    'ap': 'Android-phone',
    'aa': 'Android-all'
}

def select(stmt, params):
    with closing(conn.cursor()) as cur:
        cur.execute(stmt, params)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt
    
def mb_modify_keys(result, device_code, country_code, date):
    key = result.keys()[0] + " {device} {country} {date}".format(
        device=mb_device_mapping.get(device_code),
        country=country_code,
        date=date
    )
    value = result.values()[0]
    return {key: value}

def check_mobile_web_mobile_retention_date(date, granularity, retention_days):
    result_list = []
    result = {}
    mb_countries_list = ['us', 'gb', 'jp', 'kr', 'fr', 'de']
    for device_code in ['ip', 'ap']:
        for country_code in mb_countries_list:
            sql = """
                select sum(cnt) from plproxy.execute_select($$select count(1) as cnt from mw.domain_retention_{granularity} "
                "where date = '{date}' and device_code = '{device_code}' and country_code = '{country_code}' and retention_days={retention_days}$$)t(
                cnt bigint);""".format(
                granularity=granularity,
                device_code=device_code,
                date=date,
                country_code=country_code,
                retention_days=retention_days
            )
            rows, _ = select(sql, None)
            result[granularity] = rows
            result_list.append(mb_modify_keys(result, device_code, country_code, date))
    return result_list
    
TODAY = datetime.date.today()
start_date, last_month_end = get_month_start_end_date(get_random_day_in_last_month(datetime_to_string(TODAY)))
last_month_end = (string_to_datetime(last_month_end)).date()
month_end_before_last = (string_to_datetime(start_date) - datetime.timedelta(days=1)).date()
TODAY = datetime.date.today()
def get_retention_check_date():
    retention_dict = {}

    if TODAY.day < 12:
        retention_30_check_date = month_end_before_last + datetime.timedelta(35)
        print retention_30_check_date
        if TODAY.day < retention_30_check_date.day:
            retention_dict['days'] = 14
        else:
            retention_dict['days'] = 30
        retention_dict['check_date'] = month_end_before_last
    elif TODAY.day < 19:
        retention_dict['days'] = 7
        retention_dict['check_date'] = last_month_end
    else:
        retention_dict['days'] = 14
        retention_dict['check_date'] = last_month_end

    return retention_dict
    
print get_retention_check_date(), month_end_before_last, last_month_end

# for i in (7, 14, 30):
#     db_check_date = date - datetime.timedelta(days=i)
#     if db_check_date == last_month_end or db_check_date == month_end_before_last:
#         retention_month_date = db_check_date
#         retention_metric_date = i
mobile_web_date = check_mobile_web_mobile_retention_date(month_end_before_last, 'm', 14)
failed_results = [result.keys()[0] for result in mobile_web_date if result.values()[0][0][0] == 0]

if not failed_results:
    print('Mobile Web Domain Retention - m Check Succeed.')
else:
    print('Mobile Web Domain Retention -  Failed' + str(failed_results))
print()
                

        

In [0]:

import datetime
from random import randint
from aaintdatapipeline.application.app_qa.common.utils import get_random_day_in_last_month, datetime_to_string, \
    get_random_day_in_last_week, get_week_start_end_date, send_data_source_date_email, get_month_start_end_date, \
    string_to_datetime
    
import psycopg2
from contextlib import closing


dbinfo = {
    'NAME': 'cohort',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': 'internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com',
    'PORT': 7432,
    
}
conn = psycopg2.connect("""
            dbname={dbname} user={user} host={host} port={port} password={passwd}
            """.format(dbname=dbinfo['NAME'], user=dbinfo['USER'], passwd=dbinfo['PASSWORD'],
                       host=dbinfo['HOST'], port=dbinfo['PORT']))
                       

def select(stmt, params):
    with closing(conn.cursor()) as cur:
        cur.execute(stmt, params)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt
    

def check_mobile_web_domain_retention_led_date(date, granularity, retention_days):
    result_list = []
    result = {}
    for device_code in ['ip', 'ap']:
        mw_d_r_led = """select sum(cnt) from plproxy.execute_select($$select count(1) as cnt 
                    from c "
                    "where date = '{}' and granularity = '{}' and device_code = '{device_code}'$$)t(cnt bigint);
                """.format(date,
                           granularity,
                           device_code=device_code)
        rows, _ = select(mw_d_r_led, None)
        result[device_code + ' ' + str(date)] = rows
        result_list.append(result)
    return result_list
    

date = datetime.date.today() - datetime.timedelta(days=5) -datetime.timedelta(days=1)
retention_month_date, retention_metric_date = None, None
start_date, last_month_end = get_month_start_end_date(get_random_day_in_last_month(datetime_to_string(date)))
last_month_end = (string_to_datetime(last_month_end)).date()
month_end_before_last = (string_to_datetime(start_date) - datetime.timedelta(days=1)).date()

for i in (7, 14, 30):
    db_check_date = date - datetime.timedelta(days=i)
    if db_check_date == last_month_end or db_check_date == month_end_before_last:
        retention_month_date = db_check_date
        retention_metric_date = i
        mobile_web_date = check_mobile_web_domain_retention_led_date(last_month_end, 'm', retention_metric_date)
        # failed_results = [k for k, v in mobile_web_domain_retention_led_date.iteritems() if v[0][0] == 0]
        print(mobile_web_date)
        failed_results = [result.keys()[0] for result in mobile_web_date if result.values()[0][0][0] == 0]
        print(failed_results)
        if not failed_results:
            print('Mobile Web Domain Retention LED- m Check Succeed.')
        else:
            print('Mobile Web Domain Retention LED-  Failed' + str(failed_results))

In [0]:

from datetime import timedelta, date, datetime
from random import randint
import psycopg2
from contextlib import closing
import time


dbinfo = {
    'NAME': 'cohort',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': 'internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com',
    'PORT': 7432,
    
}
conn = psycopg2.connect("""
            dbname={dbname} user={user} host={host} port={port} password={passwd}
            """.format(dbname=dbinfo['NAME'], user=dbinfo['USER'], passwd=dbinfo['PASSWORD'],
                       host=dbinfo['HOST'], port=dbinfo['PORT']))

def select(stmt, params):
    with closing(conn.cursor()) as cur:
        cur.execute(stmt, params)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt


from aaintdatapipeline.application.app_qa.common.utils import get_random_day_in_last_month, datetime_to_string, \
    get_random_day_in_last_week, get_week_start_end_date, send_data_source_date_email, get_month_start_end_date, \
    string_to_datetime

# get date
# today = date.today()
# date = get_random_day_in_last_week(today)
# start_date, end_date = get_week_start_end_date(date)
# if today.isoweekday() < 6:
#     end_date = (string_to_datetime(start_date) - timedelta(days=1)).date()


# start_date, end_date = get_month_start_end_date(get_random_day_in_last_month(datetime_to_string(today)))
# if today.day <= 6:
#     end_date = (string_to_datetime(start_date) - datetime.timedelta(days=1)).date()

# # print(end_date)

# start_time = time.time()
# test code
start_time = time.time()
# sql: get country list
# country_list_sql = "select distinct country_code from plproxy.execute_select($$select distinct country_code from mu.city_app_m where date='2019-9-30'$$)t(country_code char) order by country_code"

# sql: get city list group by 
city_ids_sql = "select * from plproxy.execute_select($$select country_code, count(distinct(city_id)) as cnt from mu.city_app_m where date='2019-9-30' and device_code = 'ap' group by country_code$$)t(country_code char, cnt BIGINT) group by country_code, cnt order by country_code"

# print
# elapsed_time = time.time() - start_time
# print elapsed_time
# country_list, countries_count = select(country_list_sql, None)
city_ids, country_count = select(city_ids_sql, None)
print(len(city_ids))
city_count = 0
for country_city in city_ids:
    # print(country_city[1])
    city_count += country_city[1]
    
print(city_count==282, country_count==47)

# sql date in city level table 
# rows = []
# for country_code in country_list:
#     sql = """
#         select sum(cnt) from plproxy.execute_select($$select count(1) as cnt from mu.city_app_{granularity} "
#         "where date = '2019-9-30' and device_code = 'ap' and country_code = '{country_code}'$$)t(
#         cnt bigint);""".format(
#         granularity='m',
#         country_code=country_code[0]
#     )
#     row, _ = select(sql, None)
#     rows.append(row)

# for city_id in city_ids:
#     sql = """
#         select sum(cnt) from plproxy.execute_select($$select count(1) as cnt from mu.city_app_{granularity} "
#         "where date = '2019-9-30' and device_code = 'ap' and city_id = '{city_id}'$$)t(
#         cnt bigint);""".format(
#         granularity='m',
#         city_id=city_id[0]
#     )
#     row, _ = select(sql, None)
#     rows.append(row)

# elapsed_time = time.time() - start_time
# print len(rows)
    


In [0]:

import datetime
from random import randint
    
import psycopg2
from contextlib import closing


dbinfo = {
    'NAME': 'cohort',
    'USER': 'app_bdp_usage_qa',
    'PASSWORD': '2mHdFW6%#REu',
    'HOST': 'internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com',
    'PORT': 7432,
    
}
conn = psycopg2.connect("""
            dbname={dbname} user={user} host={host} port={port} password={passwd}
            """.format(dbname=dbinfo['NAME'], user=dbinfo['USER'], passwd=dbinfo['PASSWORD'],
                       host=dbinfo['HOST'], port=dbinfo['PORT']))
                       

def select(stmt, params):
    with closing(conn.cursor()) as cur:
        cur.execute(stmt, params)
        rows = cur.fetchall()
        cnt = cur.rowcount
    return rows, cnt

def check_mobile_web_led_date(date, granularity):
    result_list = []
    result = {}
    mw_d_r_led = """select sum(cnt) from plproxy.execute_select($$select count(1) as cnt from mw.domain_latest_est_date"
              " where date = '{}' and granularity = '{}' and device_code = 'ip'$$)t(cnt bigint);;
            """.format(date, granularity)
    rows, _ = select(mw_d_r_led, None)
    print(rows)
    # result[device_code + ' ' + str(date)] = rows
    # result_list.append(result)
    # return result_list
    
check_mobile_web_led_date('2019-11-30', 'w')


In [0]:

import datetime
date = datetime.date(2019, 9, 30) + datetime.timedelta(days=35)
print(date)